<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP Agent
[code](https://replit.com/@matt/OpenAI-Agents-SDK#main.py)
[video](https://www.youtube.com/watch?v=Ta5J_2KFBGM)
[replit](https://docs.replit.com/getting-started/intro-replit)
[trace](https://platform.openai.com/traces)

In [ ]:
# Install

!pip install agents
!pip install openai-agents

In [ ]:
# Imports

import os
from agents.mcp import MCPServerStdio
import asyncio
from datetime import datetime
from agents import Agent, Runner, function_tool, trace
import gradio as gr
from google.colab import userdata
from datetime import datetime

In [ ]:
# Fetch tokens for openai and pushover

openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and starts with {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

pushover_user = userdata.get("PUSHOVER_USER")
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

pushover_token = userdata.get("PUSHOVER_TOKEN")
if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
# The usual starting point

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# Make an agent with name, instructions, model

agent = Agent(name="Jokester2",
          instructions="You are a joke teller",
          model="gpt-4o-mini")

with trace("Telling a joke"):
  result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
  print(result.final_output)

In [ ]:
# The real code is here

from agents.mcp import MCPServerStdio
import asyncio
from datetime import datetime
from agents import Agent, Runner, function_tool


@function_tool
async def get_time() -> str:
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


mcp_fetch = MCPServerStdio(params={
    "command": "uvx",
    "args": ["mcp-server-fetch"],
})


async def async_main():
  async with mcp_fetch:
    agent = Agent(name="Assistant",
                  model="gpt-4.1-mini",
                  instructions="You are a helpful assistant",
                  mcp_servers=[mcp_fetch],
                  tools=[get_time])

    result = await Runner.run(
        agent, """
            Please get the content of docs.replit.com/updates and summarize them.
            Return the summary as well as the time you got the content.
            """)

    print(result.final_output)


if __name__ == "__main__":
  asyncio.run(async_main())
